In [1]:
import csv
import sys
import logging
import os
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim import corpora
import pandas as pd
import json
from gensim import corpora, models, similarities
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

D:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [57]:
def text_sim(user_text, movie_content):
    #user and movie are lists
    texts = movie_content
    dictionary = corpora.Dictionary(texts)
    
    new_doc = user_text
    new_vec = dictionary.doc2bow(user_text)
    
    corpus = [dictionary.doc2bow(text) for text in texts]
    #corpora.MmCorpus.serialize('m_text.mm', corpus)
    #corpus = corpora.MmCorpus('m_text.mm')
    tfidf = models.TfidfModel(corpus)
    doc_bow = new_vec
    corpus_tfidf = tfidf[corpus]
    
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20)
    corpus_lsi = lsi[corpus_tfidf] #bow->tfidf->fold-in-lsi
    vec_bow = new_vec
    vec_lsi = lsi[vec_bow]
    
    index = similarities.MatrixSimilarity(lsi[corpus])
    sims = index[vec_lsi]
    
    sim_id = []
    sim_rate = []
    for i, element in enumerate(sims):
        sim_id.append(i)
        sim_rate.append(element)
    similarity = dict(zip(sim_id, sim_rate))
    similarity = sorted(similarity.items(), key=lambda item:item[1], reverse=True)
    
    return similarity

In [59]:
with open('movie_data.json', 'r') as f:
    data = json.load(f)
movie_id = data['movie_index']
movie_title = data['movie_title']
movie_content = data['movie_content']

movie_data = pd.DataFrame.from_dict(data)

user_text = []
f = open('user_data.txt', encoding='utf-8')
for i in f:
    i=i.strip().split('\n')
    user_text.extend(i)
f.close()

In [68]:
s = text_sim(user_text, movie_content)

2018-08-10 16:15:03,486 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-10 16:15:03,576 : INFO : built Dictionary(9235 unique tokens: ['able', 'academy', 'according', 'action', 'active']...) from 250 documents (total 155554 corpus positions)
2018-08-10 16:15:03,641 : INFO : collecting document frequencies
2018-08-10 16:15:03,642 : INFO : PROGRESS: processing document #0
2018-08-10 16:15:03,652 : INFO : calculating IDF weights for 250 documents and 9234 features (39623 matrix non-zeros)
2018-08-10 16:15:03,677 : INFO : using serial LSI version on this node
2018-08-10 16:15:03,680 : INFO : updating model with new documents
2018-08-10 16:15:03,845 : INFO : preparing a new chunk of documents
2018-08-10 16:15:03,855 : INFO : using 100 extra samples and 2 power iterations
2018-08-10 16:15:03,857 : INFO : 1st phase: constructing (9235, 120) action matrix
2018-08-10 16:15:03,870 : INFO : orthonormalizing (9235, 120) action matrix
2018-08-10 16:15:04,047 : INFO : 2nd phas

In [69]:
#print(movie_data)
rec_list = []
for i in range(0, len(s)):
    index = s[i][0]
    print(index)
    name = movie_data['movie_title'][index]
    rec_list.append(name)

98
178
75
25
70
15
83
214
196
202
56
90
123
100
182
188
114
122
44
76
1
168
2
5
96
144
186
135
235
20
211
97
247
22
33
103
59
223
0
81
35
99
189
125
10
8
180
71
132
161
28
86
243
84
194
7
165
49
193
127
65
133
224
156
233
157
42
134
173
64
219
105
27
229
185
148
236
77
79
213
53
36
159
216
16
131
246
150
106
169
104
12
80
119
170
164
208
162
175
140
78
215
187
153
94
137
184
61
242
130
29
207
171
176
55
38
109
40
220
152
88
26
23
54
18
73
6
47
221
93
92
197
118
190
48
101
60
17
200
228
249
34
9
167
63
248
13
217
11
244
136
41
46
4
87
67
237
201
230
154
138
74
126
163
238
232
160
113
225
183
239
245
66
115
102
231
179
39
128
62
21
69
51
209
89
177
226
30
110
107
218
172
192
198
124
112
210
195
3
14
19
24
31
32
37
43
45
50
52
57
58
72
82
85
91
108
111
116
120
121
129
139
141
142
145
147
149
158
166
174
181
203
204
212
222
234
241
155
95
205
146
191
68
151
143
227
117
240
206
199


In [70]:
rec_list

['Raging Bull',
 'Rocky',
 'Taxi Driver',
 'City Lights',
 "Singin' in the Rain",
 'Goodfellas',
 'The Apartment',
 'Young Frankenstein',
 'The Night of the Hunter',
 'Annie Hall',
 'Once Upon a Time in America',
 'Some_Like_It_Hot',
 'There Will Be Blood',
 'Sherlock Jr.',
 'The Man Who Shot Liberty Valance',
 'Roman Holiday',
 'Sunrise:_A_Song_of_Two_Humans',
 'The_Deer_Hunter',
 'The Great Dictator',
 'The Sting',
 'The Godfather',
 'Butch Cassidy and the Sundance Kid',
 'The Godfather: Part II',
 "Schindler's List",
 'The Great Escape',
 'Life of Brian',
 'The Last Picture Show',
 'It_Happened_One_Night',
 'The Graduate',
 "It's a Wonderful Life",
 'Dogville',
 'Die_Hard',
 'A Christmas Story',
 'The Usual Suspects',
 'The Departed',
 'On the Waterfront',
 'Citizen Kane',
 'The Hustler',
 'The Shawshank Redemption',
 'Good_Will_Hunting',
 'Back to the Future',
 'The Third Man',
 'Anatomy of a Murder',
 'Gran Torino',
 'Forrest Gump',
 'Fight Club',
 'Touch of Evil',
 'Eternal Sunsh

In [11]:
with open('movie_data.json', 'r') as f:
    data = json.load(f)
movie_title = data['movie_title']
movie_content = data['movie_content']
#250电影数据库with id, title and content words
movie_data = pd.DataFrame.from_dict(data)